In [20]:
import pandas as pd

path = "/Users/warre/Desktop/ESADE/Python/"

reviews = pd.read_csv(path + "reviews.csv")
calendar = pd.read_csv(path + "calendar.csv")
listings = pd.read_csv(path + "listings.csv")
reviews_1k = reviews.head(1000)
calendar_1k = calendar.head(1000)
listings_1k = listings.head(1000)
reviews_1k.to_csv(path + "reviews_1000.csv", index=False)
calendar_1k.to_csv(path + "calendar_1000.csv", index=False)
listings_1k.to_csv(path + "listings_1000.csv", index=False)




In [21]:
# 2. Prepare REVIEWS dataset
# -----------------------
# Reviews have multiple rows per listing, so we aggregate

review_agg = (
    reviews.groupby("listing_id")
    .agg(
        review_count=("id", "count"),
        last_review=("date", "max"),
        first_review=("date", "min")
    )
    .reset_index()
)

# -----------------------
# 3. Merge LISTINGS + CALENDAR
#    (calendar is the base: one row per listing + date)
# -----------------------

df = calendar.merge(
    listings,
    how="left",
    left_on="listing_id",
    right_on="id"
)

# -----------------------
# 4. Merge review aggregates
# -----------------------
df = df.merge(
    review_agg,
    how="left",
    left_on="listing_id",
    right_on="listing_id"
)

# -----------------------
# 5. Optional cleanup
# -----------------------
# Drop duplicated ID column
if "id" in df.columns:
    df = df.drop(columns=["id"])

# Create a clean target (availability)
df["available_flag"] = (df["available"] == "t").astype(int)

print(df.shape)
df.head()

(2863060, 89)


,listing_id,date,available,price_x,adjusted_price,minimum_nights_x,maximum_nights_x,listing_url,scrape_id,last_scraped,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,review_count,last_review_y,first_review_y,available_flag
0,2053439,2025-09-24,f,NaN,NaN,3,30,https://www.airbnb.com/rooms/2053439,20250923203019,2025-09-24,...,f,1,1,0,0,2.19,279.0,2025-09-23,2015-04-06,0
1,2053439,2025-09-25,f,NaN,NaN,3,30,https://www.airbnb.com/rooms/2053439,20250923203019,2025-09-24,...,f,1,1,0,0,2.19,279.0,2025-09-23,2015-04-06,0
2,2053439,2025-09-26,t,NaN,NaN,3,30,https://www.airbnb.com/rooms/2053439,20250923203019,2025-09-24,...,f,1,1,0,0,2.19,279.0,2025-09-23,2015-04-06,1
3,2053439,2025-09-27,f,NaN,NaN,3,30,https://www.airbnb.com/rooms/2053439,20250923203019,2025-09-24,...,f,1,1,0,0,2.19,279.0,2025-09-23,2015-04-06,0
4,2053439,2025-09-28,f,NaN,NaN,3,30,https://www.airbnb.com/rooms/2053439,20250923203019,2025-09-24,...,f,1,1,0,0,2.19,279.0,2025-09-23,2015-04-06,0
